# Overview

The following example demonstrates a data science workflow from start to finish using real-world claims data collected by the Transportation Security Administration.

The data file used for this example can be obtained from https://support.sas.com/documentation/onlinedoc/viya/exampledatasets/TSAClaims2002_2017.csv. The data are originally from https://www.dhs.gov/tsa-claims-data.


# Load the SWAT Library and Connect to the CAS Server

Here is an example of the syntax that is used to connect to a CAS server. The LIBNAME statement is then used to assign a libref named Casuser to the Casuser caslib because a caslib cannot be used in place of a libref.

In [ ]:
import swat
# change the host and port to match your site
s = swat.CAS("cloud.example.com", 10065)

# Load the Data

There are two methods that can be used to load a data file. The first method is to load the data from a caslib (server-side load). The second method is to load the data from a location that is accessible to the CAS server but not associated with a caslib (client-side load).

## Load the Data from a Caslib

The default method of loading data is to load the data from the data source portion of a caslib, which is known as a server-side load. This requires the data file to be saved in the active caslib (Casuser). Once the file has been saved to the caslib, use the loadTable action to load the TSAClaims2002_2017.csv file into the Casuser caslib as an in-memory table.

In the casOut parameter, specify that the data is saved as a table named tsaclaims.
In the importOptions parameter, specify CSV for the fileType.
For encoding, specify latin1.
For guessRows, specify a number that includes all rows in the data set, such as 300,000, so that all rows will be scanned to determine the appropriate data type for each column.

In [ ]:
s.table.loadTable(path="TSAClaims2002_2017.csv",
                  caslib="casuser",
                  casOut={"name":"tsaclaims", 
                          "replace":True},
                  importOptions={"fileType":"CSV",
                                 "encoding":"latin1",
                                 "guessrows":30000})

## Load a Client-Side Data File from CAS

Another method of loading data into CAS memory is to load the data from an external source that is accessible to the CAS server. This example uses the HTTP procedure with the PROC CAS UPLOAD statement to perform a client-side load.

In [ ]:
result = s.upload_file("https://support.sas.com/documentation/onlinedoc/viya/exampledatasets/TSAClaims2002_2017.csv",   
                       casOut={'name':'tsaclaims',
                               'caslib':'casuser',
                               'replace':True},
                       importOptions={'fileType':'csv',
                                      'encoding':'latin1',
                                      'guessRows':300000})

# Explore the Data

## Examine the Columns

To get information about a table’s columns, use the columnInfo action on the tsaclaims table to obtain metadata about the table. The result includes the names of columns, and information about each column, including its label (if applicable), type, length, and format.

In [ ]:
s.table.columnInfo(table={"caslib":"casuser",
                          "name":"tsaclaims"})

## Examine the Rows

Preview the first few rows of the table by using the table.fetch action and specifying 10 in the to parameter. The sortBy parameter specifies that the column Close_Amount is sorted in descending order. Examine the values in the rows to identify any data issues. In the resulting table, notice that the values in the two date columns (Date_Received and Incident_Date) are 5-digit numbers and are not properly formatted in an MM-DD-YYYY format. In addition, the Close_Amount variable shows values with a missing dollar sign. The columns need to be modified to apply the appropriate date and currency formats.

In [ ]:
s.table.fetch(table={"caslib":"casuser",
                      "name":"tsaclaims"},
              to=10,
              sortBy=[{"name":"Close_Amount", "order":"DESCENDING"}])

# Prepare the Data

## Add Formats to Variables

The next step is to add formats to the variables. Use the table.alterTable action to rename columns and apply formats to the date and amount columns. Assign new column labels to the column names so that the underscores are removed. Assign the format dollar19.2 to Close_Amount to display the formatted values with a dollar sign and two decimal places. Assign a format of mmddyy10. to the Date_Received and Incident_Date columns to display the values in the format of MM-DD-YYYY.

In [ ]:
s.table.alterTable(name="tsaclaims",
                   caslib="casuser",
                   columns=[
                           {"name":"Close_Amount", "label":"Close Amount", "format":"dollar19.2"},
                           {"name":"Claim_Number", "label":"Claim Number"},
                           {"name":"Date_Received", "label":"Date Received", "format":"mmddyy10."},
                           {"name":"Incident_Date", "label":"Incident Date", "format":"mmddyy10."},
                           {"name":"Airport_Code", "label":"Airport Code"},
                           {"name":"Airport_Name", "label":"Airport Name"},
                           {"name":"Claim_Type", "label":"Claim Type"},
                           {"name":"Claim_Site", "label":"Claim Site"},
                           {"name":"Item_Category", "label":"Item Category"}])

s.table.columnInfo(table="tsaclaims")

The resulting table shows that the Date_Received and Incident_Date columns now specify the format of MMDDYY, and the Close_Amount column now specifies the format of DOLLAR.

## Count the Number of Unique and Missing Column Values

Examine the number of unique and missing values for each variable to determine whether further cleaning is needed. Run the simple.distinct action to identify the number of distinct values and the number of missing values for each column.

In [ ]:
s.simple.distinct(table="tsaclaims")

## Remove Duplicate Rows

Full duplicate rows that have duplicate values on all variables should be removed. The deduplicate action removes rows that contain duplicated values on the variables listed in the groupBy parameter.

In the groupBy parameter, it is necessary to list all variables in the table if you want to remove full duplicate rows (that is, rows that have duplicate values on all variables).
The casOut parameter specifies that the output table containing rows with duplicates removed is named tsaclaims_nodups.
The duplicateOut parameter specifies that the output table containing the duplicate rows that were removed is named tsa_claims_dups.

In [ ]:
s.builtins.loadActionSet(actionSet="deduplication")

s.deduplication.deduplicate(
                            table={"caslib":"casuser",
                                   "name":"tsaclaims",
                                   "groupBy":[{"name":"Claim_Number"}, 
                                              {"name":"Date_Received"}, 
                                              {"name":"Incident_Date"}, 
                                              {"name":"Airport_Code"}, 
                                              {"name":"Airport_Name"}, 
                                              {"name":"Claim_Type"}, 
                                              {"name":"Claim_Site"}, 
                                              {"name":"Item_Category"}, 
                                              {"name":"Close_Amount"}, 
                                              {"name":"Disposition"}, 
                                              {"name":"StateName"}, 
                                              {"name":"State"}, 
                                              {"name":"County"}, 
                                              {"name":"City"}]
                                  },
                             casOut={"caslib":"casuser", 
                                     "name":"tsaclaims_nodups", 
                                     "replace":True},
                             duplicateOut={"caslib":"casuser", 
                                           "name":"tsaclaims_dups", 
                                           "replace":True},
                             noDuplicateKeys=True)

## Identify Data Quality Issues

A frequency distribution can be used to examine the columns for data issues. The simple.freq action generates a frequency distribution showing the counts of each unique value for each of the variables listed in the inputs parameter. The columns Date_Received and Incident_Date are assigned the format year4. to display the values with a 4-digit year. Columns with options specified (such as a format) need to be enclosed in braces.

In [ ]:
s.simple.freq(table={"caslib":"casuser", 
                     "name":"tsaclaims_nodups"},
              inputs=[{"name":"Claim_Site"},
                      {"name":"Disposition"},
                      {"name":"Claim_Type"},
                      {"name":"Date_Received", "format":"year4."},
                      {"name":"Incident_Date", "format":"year4."}])

The data contains some errors including missing values, inconsistent values and out-of-range values that can be cleaned using the table.update action.

## Replace Missing and Inconsistent Values

Clean the Claim_Site and Claim_Type columns using table.update. Replace the inconsistent values in the columns so that there are no missing values, misspelled values, or values with multiple claim types.

The first two update actions selects rows in the Claim_Site and Claim_Type columns where the value is null or - and replaces the values with 'Unknown'.
The third update action selects rows where the Claim_Type value is 'Passenger Property Loss/Personal Injury', or 'Passenger Property Loss/Personal Injur' and replaces the value of Claim_Type with 'Passenger Property Loss'.
The fourth update action selects rows where the Claim_Type value is 'Property Damage/Personal Injury' and replaces the values of Claim_Type with 'Property Damage'.

In [ ]:
s.table.update(table={"name":"tsaclaims_nodups", 
                      "caslib":"casuser", 
                      "where":"Claim_Site is null or Claim_Site='-'"},
                 set=[{"var":"Claim_Site", "value":"'Unknown'"}])

s.table.update(table={"name":"tsaclaims_nodups", 
                      "caslib":"casuser", 
                      "where":"Claim_Type is null or Claim_Type='-'"},
                 set=[{"var":"Claim_Type", "value":"'Unknown'"}])
               
s.table.update(table={"name":"tsaclaims_nodups", 
                      "caslib":"casuser", 
                      "where":"""Claim_Type='Passenger Property Loss/Personal Injury' 
                              or Claim_Type='Passenger Property Loss/Personal Injur'"""},
                 set=[{"var":"Claim_Type", "value":"'Passenger Property Loss'"}])

s.table.update(table={"name":"tsaclaims_nodups", 
                      "caslib":"casuser", 
                      "where":"Claim_Type='Property Damage/Personal Injury'"},
                 set=[{"var":"Claim_Type", "value":"'Property Damage'"}])

Clean the Disposition column with three table.update actions.

The first update action selects rows where the value of Disposition is null or and replaces the values of Disposition with 'Unknown'.
The second update action selects rows where the value of Disposition is "Closed: Canceled" and replaces the values of Disposition with "Closed:Canceled".
The third update action selects rows where the value of Disposition is "losed:Contractor Claim" and replaces the values of Disposition with "Closed:Contractor Claim".

In [ ]:
s.table.update(table={"name":"tsaclaims_nodups", 
                      "caslib":"casuser", 
                      "where":"Disposition is null or Disposition='-'"},
                 set=[{"var":"Disposition", "value":"'Unknown'"}])

s.table.update(table={"name":"tsaclaims_nodups", 
                      "caslib":"casuser", 
                      "where":"Disposition='Closed: Canceled'"},
                 set=[{"var":"Disposition", "value":"'Closed:Canceled'"}])

s.table.update(table={"name":"tsaclaims_nodups", 
                      "caslib":"casuser", 
                      "where":"Disposition='losed: Contractor Claim'"},
                 set=[{"var":"Disposition", "value":"'Closed:Contractor Claim'"}])

The values of the StateName column should be converted from uppercase to propercase. Use the table.update action with the propcase function to convert all values of a column to proper case.

In [ ]:
s.table.update(table={"caslib":"casuser", 
                      "name":"tsaclaims_nodups"},
                 set=[{"var":"StateName", "value":"propcase(StateName)"}])

To create tables to examine the cleaned columns, use a simple.freq action specifying the columns as inputs.

In [ ]:
s.simple.freq(table={"caslib":"casuser", 
                     "name":"tsaclaims_nodups"},
              inputs=[{"name":"Claim_Site"},
                      {"name":"Disposition"},
                      {"name":"Claim_Type"},
                      {"name":"Date_Received", "format":"year4."},
                      {"name":"Incident_Date", "format":"year4."}])

s.simple.freq(table={"caslib":"casuser", 
                     "name":"tsaclaims_nodups"},
              inputs=[{"name":"State"},
                      {"name":"StateName"}])

## Add a New Column to Identify Rows with Errors

Create a new column to identify rows that have date values that are out of range (earlier than 2002 or later than 2017). These rows will require further investigation. In the computedVars parameter, specify the name of the new column as Date_Issues. In the computedVarsProgram parameter, write a query to assign a value of ‘Needs Review’ to the Date_Issues variable if the year of the Incident_Date or Date_Received variable falls outside of the range between 2002 and 2017.

In [ ]:
s.table.copyTable(casout={"caslib":"casuser", 
                          "name":"tsaclaims_cleaned", 
                          "replace":True},
                   table={"caslib":"casuser", 
                          "name":"tsaclaims_nodups",
                          "computedVars":[{"name":"Date_Issues", 
                                           "label":"Date Issues"}],
                          "computedVarsProgram":"""if year(Incident_Date) < 2002 or 
                                                   year(Incident_Date) > 2017 or 
                                                   year(Date_Received) < 2002 or 
                                                   year(Date_Received) > 2017 
                                                   then Date_Issues='Needs Review'"""})

s.simple.freq(table={"caslib":"casuser", 
                     "name":"tsaclaims_cleaned"},
              inputs={"Date_Issues"})

## Drop Columns

Columns that are no longer needed can be dropped using the table.alterTable action. Here the columns County and City are specified in the drop parameter.

In [ ]:
s.table.alterTable(caslib="casuser",
                   name="tsaclaims_cleaned",
                   drop={"City", "County"})

## Examine the Cleaned Data

Examine the cleaned data using the columnInfo and fetch actions.

In [ ]:
s.table.columnInfo(table="tsaclaims_cleaned")

In [ ]:
s.table.fetch(table={"caslib":"casuser", 
                     "name":"tsaclaims_cleaned"})

# Analyze the Data

## Create Frequency Tables 

The simple.freq action can be used to perform analysis to examine the number of date issues and the number of claims per year of Incident_Date in the overall data. The output of the second simple.freq action is saved as a table named tsaclaims_cleaned_freq_date.

In [ ]:
s.simple.freq(table={"caslib":"casuser", 
                     "name":"tsaclaims_cleaned"},
              inputs=[{"name":"Date_Issues"}])

s.simple.freq(table={"caslib":"casuser", 
                     "name":"tsaclaims_cleaned"},
              inputs=[{"name":"Incident_Date",
                       "format":"year4."}],
              casOut={"caslib":"casuser",
                      "name":"tsaclaims_cleaned_freq_date",
                      "replace":True})

## Generate Frequencies and Summary Statistics on a Subset of Data

Here the simple.freq action is used to generate frequency tables to show the number of claims for each Claim_Type, Claim_Site, and Disposition, as well as to calculate the mean, minimum, maximum, and sum of Close_Amount for claims in Hawaii that do not have date issues. The output of the first simple.freq action is saved as a table named tsaclaims_cleaned_freq_date.

In [ ]:
s.simple.freq(table={"caslib":"casuser", 
                     "name":"tsaclaims_cleaned",
                     "where":"StateName='Hawaii' and Date_Issues is null"},
              inputs=[{"name":"Claim_Site"},
                      {"name":"Claim_Type"},
                      {"name":"Disposition"}],
              casOut={"caslib":"casuser",
                      "name":"tsaclaims_cleaned_freq_vars",
                      "replace":True})

s.simple.summary(table={"caslib":"casuser", 
                        "name":"tsaclaims_cleaned",
                        "where":"StateName='Hawaii' and Date_Issues is null"},
                inputs=[{"name":"Close_Amount"}],
                subSet={"N", "MEAN", "MIN", "MAX", "SUM"})

# Visualize the Data 

## Create Bar Charts to Visualize the Results

The pandas plot.bar method can be used to visualize data that has been preprocessed using CAS actions. This method uses the matplotlib library through pandas rather than using matplotlib functions directly which creates cleaner plots. The method is used here to create a bar chart that shows the number of claims by year of incident date.

### By Year

In [ ]:
tsaSubset = s.table.fetch(fetchVars=[{"name":"_Fmtvar_"}, 
                                     {"name":"_Frequency_"}], 
                          table={"caslib":"casuser", 
                                 "name":"tsaclaims_cleaned_freq_date",
                                 "where":"_Numvar_"},
                          sortBy=[{"name":"_Fmtvar_", 
                                   "order":"ASCENDING"}])

tsa_df = tsaSubset["Fetch"]
tsa_df
tsaTableYearVFreq = tsa_df.plot.bar(x="_Fmtvar_", y="_Frequency_")
tsaTableYearVFreq.set_xlabel("Incident Date (Year)")
tsaTableYearVFreq.set_ylabel("Number of Claims")
tsaTableYearVFreq.set_title("Number of Claims by Year")

Here the pandas plot.bar method is used multiple times to create bar charts showing the number of claims by claim site, claim type, and disposition.

### By Claim Site

In [ ]:
tsaSubsetClaimSite = s.table.fetch(fetchVars=[{"name":"_Fmtvar_"}, 
                                              {"name":"_Frequency_"}], 
                                   table={"caslib":"casuser", 
                                          "name":"tsaclaims_cleaned_freq_vars",
                                          "where":"_Column_='Claim_Site'"},
                                   sortBy=[{"name":"_Frequency_", 
                                            "order":"DESCENDING"}])

tsaSubsetClaimSite.keys()
tsa_df_Claim_Site = tsaSubsetClaimSite["Fetch"]
tsa_df_Claim_Site

tsaTableClaimSiteVFreq = tsa_df_Claim_Site.plot.bar(x="_Fmtvar_", y="_Frequency_")
tsaTableClaimSiteVFreq.set_xlabel("Claim Site")
tsaTableClaimSiteVFreq.set_ylabel("Number of Claims")
tsaTableClaimSiteVFreq.set_title("Number of Claims by Claim Site in Hawaii")

### By Claim Type

In [ ]:
tsaSubsetClaimType = s.table.fetch(fetchVars=[{"name":"_Fmtvar_"}, 
                                              {"name":"_Frequency_"}], 
                                   table={"caslib":"casuser", 
                                          "name":"tsaclaims_cleaned_freq_vars",
                                          "where":"_Column_='Claim_Type'"},
                                   sortBy=[{"name":"_Frequency_", 
                                            "order":"DESCENDING"}])

tsaSubsetClaimType.keys()
tsa_df_Claim_Type = tsaSubsetClaimType["Fetch"]
tsa_df_Claim_Type

tsaTableClaimTypeVFreq = tsa_df_Claim_Type.plot.bar(x="_Fmtvar_", y="_Frequency_")
tsaTableClaimTypeVFreq.set_xlabel("Claim Type")
tsaTableClaimTypeVFreq.set_ylabel("Number of Claims")
tsaTableClaimTypeVFreq.set_title("Number of Claims by Claim Type in Hawaii")

### By Disposition

In [ ]:
tsaSubsetDisposition = s.table.fetch(fetchVars=[{"name":"_Fmtvar_"}, 
                                                {"name":"_Frequency_"}], 
                                     table={"caslib":"casuser", 
                                            "name":"tsaclaims_cleaned_freq_vars",
                                            "where":"_Column_='Disposition'"},
                                     sortBy=[{"name":"_Frequency_", 
                                              "order":"DESCENDING"}])

tsaSubsetDisposition.keys()
tsa_df_Disposition = tsaSubsetDisposition["Fetch"]
tsa_df_Disposition

tsaTableDispositionVFreq = tsa_df_Disposition.plot.bar(x="_Fmtvar_", y="_Frequency_")
tsaTableDispositionVFreq.set_xlabel("Disposition")
tsaTableDispositionVFreq.set_ylabel("Number of Claims")
tsaTableDispositionVFreq.set_title("Number of Claims by Disposition in Hawaii")